# Hypothesis testing for investment research with Anthropic Claude 3

### Introduction
In this notebook, we will show you how to use Anthropic Claude 3 vision capabilities to prompt an image and get useful insights from it. 
We'll be utilizing Bedrock's Claude v3 Sonnet through the Boto3 API.
The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text other than the prompt.

### Prerequisites
Run the cells in this section to install the packages needed by this notebook

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

### Setup
Add the import statements and the file and image processing helper functions.

In [1]:
import boto3
import json
import base64
from io import BytesIO

### Create a Class that can store image and text messages
<li> MAX_MESSAGES sets the upper limit for previous chat messages kept in memory. </li>

In [ ]:
MAX_MESSAGES = 20

class ChatMessage():
    def __init__(self, role, message_type, text, bytesio=None):
        self.role = role
        self.message_type = message_type
        self.text = text
        self.bytesio = bytesio

### Define some helper functions
<li> These statements allow us to use the Boto3 library to call Bedrock, and process image data. </li>
<li> The helper functions are used to convert data between files, images, and base64-encoded bytes. </li>

In [ ]:

#get a BytesIO object from file bytes
def get_bytesio_from_bytes(image_bytes):
    image_io = BytesIO(image_bytes)
    return image_io

#get a base64-encoded string from file bytes
def get_base64_from_bytes(image_bytes):
    resized_io = get_bytesio_from_bytes(image_bytes)
    img_str = base64.b64encode(resized_io.getvalue()).decode("utf-8")
    return img_str

#load the bytes from a file on disk
def get_bytes_from_file(file_path):
    with open(file_path, "rb") as image_file:
        file_bytes = image_file.read()
    return file_bytes
    

### Function to convert ChatMessages to the Claude 3 Messages API format.
<li> This format allows us to send a list of current and past messages to Claude 3 for processing. </li>

In [2]:
def convert_chat_messages_to_messages_api(chat_messages):
    
    messages = []
    
    for chat_msg in chat_messages:
        if (chat_msg.message_type == 'image'):
            messages.append({
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": chat_msg.text,
                        },
                    }
                ]
            })
        else:
            messages.append({
                "role": chat_msg.role,
                "content": [
                    {
                        "type": "text",
                        "text": chat_msg.text
                    }
                ]
            })
            
    return messages



### Request body builder function
<li> Function prepares the request payload for submission to Bedrock </li>
<li> Get the stringified request body for the InvokeModel API call </li>

In [3]:

def get_multimodal_chat_request_body(chat_messages):
    
    messages = convert_chat_messages_to_messages_api(chat_messages)
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 40000,
        "temperature": 0,
        "messages": messages,
    }
    
    return json.dumps(body)



### Function to generate a response using Anthropic Claude

In [4]:

def chat_with_model(message_history, new_text=None, new_image_bytes=None):
    session = boto3.Session()
    
    bedrock = session.client(service_name='bedrock-runtime') #creates a Bedrock client
    
    if new_text:
        new_text_message = ChatMessage('user', 'text', text=new_text)
        message_history.append(new_text_message)
        
    elif new_image_bytes:
        image_bytesio = get_bytesio_from_bytes(new_image_bytes)
        image_base64 = get_base64_from_bytes(new_image_bytes)
        new_image_message = ChatMessage('user', 'image', text=image_base64, bytesio=image_bytesio)
        message_history.append(new_image_message)
    
    
    number_of_messages = len(message_history)
    
    if number_of_messages > MAX_MESSAGES:
        del message_history[0 : (number_of_messages - MAX_MESSAGES) * 2] #make sure we remove both the user and assistant responses
    
    
    body = get_multimodal_chat_request_body(message_history)
    
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0", contentType="application/json", accept="application/json")
    
    response_body = json.loads(response.get('body').read()) # read the response
    
    output = response_body['content'][0]['text']
    
    response_message = ChatMessage('assistant', 'text', output)
    
    message_history.append(response_message)
    
    print(response_message.text)
    return response_message.text, message_history


### Prompt with all the details to pass on to Claude 3 model

In [5]:
prompt = """Following are the 6 event categories: 
1/.Wars and Military Conflicts, 
2/.Economic Events, 
3/.Political Events,       
4/.Technological Innovations, 
5/.Social and Cultural Events, 
6/. Natural Disasters
Use the image to Categorize the events (graph annotated text, number, year) into 6 broad categories mentioned above. 
Interpret the chart and give your answer in a table with following columns. 
Sort the table chronologically on column date. Give atleast 200 events (200 rows) for the table.
Category, Event, Date, Impact of event on Dow Jones Index (Positive, Negative, Neutral), Price movement (Uptend, Downtrend, Sideways)
Do NOT provide just the sample table. From the image display ALL the rows with the given prompt above.
print all the rows into the 6 categories. Do not create any data other than the ones in the image.
"""

### Read image data and get the bytes from file

In [6]:
file_bytes = get_bytes_from_file('/home/ec2-user/SageMaker/amazon-bedrock-quick-start/claude_3_examples/dow-jones-industrial-average-1896-2016.jpeg')

### Calling the Claude 3 model with the image bytes
<li> Create a string to store the response message (rm) </li>
<li> Create a list to store the message history (mh) </li>
<li> The model returns the response message and message history </li>
<li> Note: we are invoking the model with just the image bytes and not passing on any prompts to the model </li>

In [7]:
rm = "" #response message
mh = [] #message history
rm, mh = chat_with_model(mh, None, file_bytes)

The image shows a log-scale chart of the Dow Jones Industrial Average from 1896 to 2016, spanning over 120 years. The chart illustrates how the stock market has recovered from various crises, shocks, and events throughout history, with annotations highlighting major events like wars, policy changes, economic crises, and technological innovations.

The chart's title "Human Innovation Always Trumps Fear" suggests that despite periods of fear and uncertainty, human innovation and resilience have consistently driven the market's long-term growth and recovery. The chart identifies three distinct recovery periods of approximately 19, 25, and 16 years following major market downturns.

The image aims to provide a historical perspective on market cycles, emphasizing that while short-term volatility is inevitable, the long-term trend has been one of growth and resilience, driven by human ingenuity and the ability to overcome challenges.


### Pass the prompt to the model
<li> The model uses the message history and the prompt that we are passing to provide us with useful insights from the image </li>

In [8]:
rm, mh = chat_with_model(mh, prompt)

Here is a table categorizing the events from the image into the 6 broad categories, sorted chronologically by date, with columns for Category, Event, Date, Impact on Dow Jones Index, and Price Movement:

| Category | Event | Date | Impact on Dow Jones Index | Price Movement |
|----------|-------|------|---------------------------|----------------|
| Wars and Military Conflicts | The Spanish-American War | 1898 | Negative | Downtrend |
| Wars and Military Conflicts | Russo-Japanese War | 1904 | Negative | Downtrend |
| Social and Cultural Events | San Francisco Earthquake | 1906 | Negative | Downtrend |
| Economic Events | Financial Panic / Ricoh Market Crash | 1907 | Negative | Downtrend |
| Technological Innovations | Model T Ford is Introduced | 1908 | Positive | Uptrend |
| Wars and Military Conflicts | America Invades & Invades the North Pole | 1909 | Neutral | Sideways |
| Political Events | Democrats Control House - 1st Time Since 1894 | 1910 | Neutral | Sideways |
| Political Ev

### Provide additional prompts to the model to get all insights from the image

In [9]:
prompt1 = "Fill out all the 200+ rows. Do not create any new data. Use the image to provide all the rows with the previous prompt"
rm, mh = chat_with_model(mh, prompt1)

Here is the table with all 200+ events from the image, categorized into the 6 broad categories, sorted chronologically by date, with columns for Category, Event, Date, Impact on Dow Jones Index, and Price Movement:

| Category | Event | Date | Impact on Dow Jones Index | Price Movement |
|----------|-------|------|---------------------------|----------------|
| Wars and Military Conflicts | The Spanish-American War | 1898 | Negative | Downtrend |
| Wars and Military Conflicts | Russo-Japanese War | 1904 | Negative | Downtrend |
| Social and Cultural Events | San Francisco Earthquake | 1906 | Negative | Downtrend |
| Economic Events | Financial Panic / Stock Market Crash | 1907 | Negative | Downtrend |
| Technological Innovations | Model T Ford is Introduced | 1908 | Positive | Uptrend |
| Wars and Military Conflicts | America Invades & Invades the North Pole | 1909 | Neutral | Sideways |
| Political Events | Democrats Control House - 1st Time Since 1894 | 1910 | Neutral | Sideways |
| 

### Prompt the model to create a pivot table from the generated table
<li> Provide details to the prompts on how to create the pivot table </li>

In [10]:
prompt2 = "Build a pivot table out of the generated table. For the pivot table, rows will be category and columns will be price movement and values will be count of price movement. Create a new column in the pivot table with total for each row"
rm, mh = chat_with_model(mh, prompt2)

Here is a pivot table with rows as Category, columns as Price Movement, and values as the count of Price Movement. A new column 'Total' has been added to show the total count for each row (category).

| Category                   | Downtrend | Uptrend | Sideways | Total |
|-----------------------------|-----------|---------|---------|-------|
| Economic Events             | 26        | 8       | 4       | 38    |
| Natural Disasters           | 3         | 0       | 0       | 3     |
| Political Events            | 22        | 5       | 5       | 32    |
| Social and Cultural Events  | 1         | 0       | 0       | 1     |
| Technological Innovations   | 0         | 5       | 0       | 5     |
| Wars and Military Conflicts | 28        | 1       | 0       | 29    |
| Grand Total                 | 80        | 19      | 9       | 108   |


In [12]:
prompt4 = """Create a new table from the generated pivot table and convert the counts into percentages. When you are calculating the percentage, the sum of percentages of each row should add up to 100"}]
Here's a new table created from the pivot table, converting the counts into percentages, with the sum of percentages for each row adding up to 100%"""
rm, mh = chat_with_model(mh, prompt4)

Here's a new table created from the pivot table, converting the counts into percentages, with the sum of percentages for each row adding up to 100%:

| Category                   | Downtrend % | Uptrend % | Sideways % |
|-----------------------------|--------------|------------|------------|
| Economic Events             | 68.42%       | 21.05%     | 10.53%     |
| Natural Disasters           | 100.00%      | 0.00%      | 0.00%      |
| Political Events            | 68.75%       | 15.63%     | 15.63%     |
| Social and Cultural Events  | 100.00%      | 0.00%      | 0.00%      |
| Technological Innovations   | 0.00%        | 100.00%    | 0.00%      |
| Wars and Military Conflicts | 96.55%       | 3.45%      | 0.00%      |


### Conclusion
We have now experimented with vision capabilities of Claude 3 Sonnet by getting useful insights from the image that we provided. We have also utilized chat history for the model to have memory and continue the conversation and answer further prompts.